In [3]:
import requests
import json
import pandas as pd
from pprint import pprint
import pickle

base_url = "https://api.openalex.org/"
email = "katz.562@osu.edu"


def pickle_dump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)


dropcolumns = ['orcid', 'display_name_alternatives', 'relevance_score', 'summary_stats', 'ids',
               'affiliations', 'last_known_institutions', 'topics', 'topic_share', 'updated_date', 'created_date']
with open('data/authors.jsonl', 'r') as f:
    df = pd.read_json(f, lines=True)
# author_list = []
df = df.drop(columns=dropcolumns)

In [ ]:
import pandas as pd

# Read JSON lines file
df = pd.read_json('data/authors.jsonl', lines=True)

# Drop unnecessary columns directly
df.drop(columns=[
    'orcid', 'display_name_alternatives', 'relevance_score', 'summary_stats', 
    'ids', 'affiliations', 'last_known_institutions', 'topics', 
    'topic_share', 'updated_date', 'created_date'
], inplace=True)

# Modify 'id' column in place
df['id'] = df['id'].apply(lambda x: str(x).split('/')[-1])

# Create DataFrames directly and rename columns in one step
cited_by_count_df = df[['id', 'cited_by_count']].rename(columns={'id': 'author_id'})
works_api_url_df = df[['id', 'works_api_url']].rename(columns={'id': 'author_id'})
display_name_df = df[['id', 'display_name']].rename(columns={'id': 'author_id'})

# Unpack the concepts directly and rename column
df['x_concepts'] = df['x_concepts'].apply(lambda x: [concept['display_name'] for concept in x])
x_concepts_df = df[['id', 'x_concepts']].explode('x_concepts').rename(columns={'id': 'author_id'})

# Process counts by year and rename columns
counts_by_year_df = df[['id', 'counts_by_year']].explode('counts_by_year')
counts_by_year_df.dropna(subset=['counts_by_year'], inplace=True)
counts_by_year_df = counts_by_year_df.assign(
    year=counts_by_year_df['counts_by_year'].apply(lambda x: x['year']),
    works_count=counts_by_year_df['counts_by_year'].apply(lambda x: x['works_count']),
    cited_by_count=counts_by_year_df['counts_by_year'].apply(lambda x: x['cited_by_count'])
).drop(columns='counts_by_year').rename(columns={'id': 'author_id'})

# Read gender data and rename columns
gender_df = pd.read_json('data/predicted_gender.jsonl', lines=True).rename(columns={'id': 'author_id'})


In [ ]:
df = pd.read_json('data/works.jsonl', lines=True)

In [ ]:
cols = ['id', 'display_name', 'publication_date', 'open_access']
works_df = df[cols]
works_df['publication_date'] = pd.to_datetime(
    works_df['publication_date'], errors='coerce')

In [ ]:
authorships_df = df[['id', 'authorships']].explode('authorships')

In [ ]:
authorships_df['author_id'] = authorships_df['authorships'].apply(
    lambda x: x['author']['id'] if x['author'] is not None else None)
authorships_df['author_display_name'] = authorships_df['authorships'].apply(
    lambda x: x['author']['display_name'] if x['author'] is not None else None)
authorships_df.drop(columns='authorships', inplace=True)

In [ ]:
works_df.rename(columns={'id': 'work_id',
                         'display_name': 'work_display_name'}, inplace=True)
authorships_df.rename(columns={'id': 'work_id'}, inplace=True)

In [ ]:
works_df['work_id'] = works_df['work_id'].apply(
    lambda x: str(x).split('/')[-1])

KeyError: 'work_id'

In [ ]:
authorships_df['work_id'] = authorships_df['work_id'].apply(
    lambda x: str(x).split('/')[-1])

In [ ]:
# List of DataFrames
dataframes = [cited_by_count_df, works_api_url_df,
              display_name_df, x_concepts_df, counts_by_year_df]

# Iterate over each DataFrame
for df in dataframes:
    # Find the variable name by checking each item in globals()
    df_name = [name for name, value in globals().items() if value is df][0]

    # Save the DataFrame to a JSONL file using the variable name
    df.to_json(
        f"data/{df_name}.jsonl",
        orient="records",
        lines=True
    )

NameError: name 'cited_by_count_df' is not defined

In [ ]:
df['x_concepts'] = df['x_concepts'].apply(
    lambda x: [concept['display_name'] for concept in x])
x_concepts_df = df[['id', 'x_concepts']].explode('x_concepts')

In [ ]:
counts_by_year_df = df[['id', 'counts_by_year']].explode('counts_by_year')
counts_by_year_df = counts_by_year_df[counts_by_year_df['counts_by_year'].notna(
)]
counts_by_year_df['year'] = counts_by_year_df['counts_by_year'].apply(
    lambda x: x['year'])
counts_by_year_df['works_count'] = counts_by_year_df['counts_by_year'].apply(
    lambda x: x['works_count'])
counts_by_year_df['cited_by_count'] = counts_by_year_df['counts_by_year'].apply(
    lambda x: x['cited_by_count'])

counts_by_year_df.drop(columns='counts_by_year', inplace=True)

In [4]:
works_df = pd.read_json('works.jsonl', lines=True)
works_df

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,referenced_works_count,referenced_works,related_works,abstract_inverted_index,abstract_inverted_index_v3,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated
0,https://openalex.org/W4382515348,https://doi.org/10.53621/jippmas.v3i1.230,Penyuluhan dan Pelatihan Pembuatan Kompos Orga...,Penyuluhan dan Pelatihan Pembuatan Kompos Orga...,2023,2023-06-29,{'openalex': 'https://openalex.org/W4382515348...,id,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,18,"[https://openalex.org/W2773903859, https://ope...","[https://openalex.org/W54078636, https://opena...","{'Peningkatan': [0, 12], 'produksi': [1, 46], ...",NaN,https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2024, 'cited_by_count': 1}]",2025-02-20T07:15:03.308110,2023-06-30,NaN
1,https://openalex.org/W4400300197,https://doi.org/10.30872/abdikesmasmulawarman....,Peningkatan Pengetahuan Ibu Melalui Penyuluhan...,Peningkatan Pengetahuan Ibu Melalui Penyuluhan...,2023,2023-11-01,{'openalex': 'https://openalex.org/W4400300197...,id,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,0,[],"[https://openalex.org/W4389568370, https://ope...","{'Indonesia': [0], 'termasuk': [1], 'salah': [...",NaN,https://api.openalex.org/works?filter=cites:W4...,[],2025-02-20T04:44:57.202280,2024-07-05,NaN
2,https://openalex.org/W4302328439,None,Polymorphism at codon 36 of the p53 gene.,Polymorphism at codon 36 of the p53 gene.,1994,1994-01-01,{'openalex': 'https://openalex.org/W4302328439...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,0,[],"[https://openalex.org/W3202592045, https://ope...",None,NaN,https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2022, 'cited_by_count': 2}, {'year':...",2025-02-22T15:05:19.874221,2022-10-06,NaN
3,https://openalex.org/W4391971651,https://doi.org/10.21203/rs.3.rs-3894407/v1,Facile synthesis of KOH and ball milling co-mo...,Facile synthesis of KOH and ball milling co-mo...,2024,2024-02-20,{'openalex': 'https://openalex.org/W4391971651...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",preprint,...,68,"[https://openalex.org/W1979515203, https://ope...","[https://openalex.org/W4206968271, https://ope...","{'Abstract': [0], 'In': [1], 'this': [2, 160],...",NaN,https://api.openalex.org/works?filter=cites:W4...,[],2025-02-26T06:14:54.642644,2024-02-21,NaN
4,https://openalex.org/W4403955429,https://doi.org/10.2139/ssrn.5005706,Facile Synthesis of Ball Milling and Koh Co-Mo...,Facile Synthesis of Ball Milling and Koh Co-Mo...,2024,2024-01-01,{'openalex': 'https://openalex.org/W4403955429...,en,"{'is_oa': False, 'landing_page_url': 'https://...",preprint,...,0,[],"[https://openalex.org/W4402069804, https://ope...",None,NaN,https://api.openalex.org/works?filter=cites:W4...,[],2025-03-02T18:52:09.578052,2024-11-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374086,https://openalex.org/W1450575663,None,Evaluation of mango (Mangifera indica L.) vari...,Evaluation of mango (Mangifera indica L.) vari...,2015,2015-01-01,{'openalex': 'https://openalex.org/W1450575663...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,0,[],"[https://openalex.org/W3097135428, https://ope...",None,NaN,https://api.openalex.org/works?filter=cites:W1...,[],2025-01-19T18:24:06.746968,2016-06-24,NaN
374087,https://openalex.org/W2622517429,None,"Effect of speciality fertilizers on quality, y...","Effect of speciality fertilizers on quality, y...",2016,2016-01-01,{'openalex': 'https://openalex.org/W2622517429...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,0,[],"[https://openalex.org/W932318135, https://open...",None,NaN,https://api.openalex.org/works?filter=cites:W2...,[],2025-01-22T15:27:50.002737,2017-06-15,NaN
374088,https://openalex.org/W3142012177,https://doi.org/10.17660/actahortic.2015.1066.14,STUDIES ON WATER BALA

In [ ]:
import os
import logging
from openai import OpenAI
from pprint import pprint
# Initialize the OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def get_gender_predictions(names_list):
    """
    Takes a list of names and returns a list of gender predictions.

    :param names_list: List of names (strings) to predict genders for.
    :return: List of predicted genders corresponding to each name.
    """
    # Join the names into a single string, separated by commas
    input_string = ", ".join(names_list)

    # Create a single response for the list of names
    response = client.responses.create(
        model='gpt-3.5-turbo',

        instructions=(
            'You are a helpful assistant that determines the most likely gender of a list of persons with specified names. '
            'Respond with a comma-separated list of genders corresponding to each name: "Male", "Female", or "Unknown".'
        ),
        input=f"Names: {input_string}."
    )

    # Log the response
    logging.info(f"Response: {response}")
    # Extract the response text and split it into a list of genders
    genders = response.output_text.strip().split(", ")

    return genders


# # Example usage
names = ["Alice", "Bob", "Charlie"]
genders = get_gender_predictions(names)

In [ ]:
import re
import glob


try:
    gender_df = pd.read_json('data/predicted_gender.jsonl', lines=True)
except FileNotFoundError:
    df['strip_display_name'] = df['display_name'].str.replace(
        '\.\s?', ' ', regex=True)

    # Filter the DataFrame
    filtered_df = df[~df['strip_display_name'].str.match(
        r'^\b\w{1,2}\b ')]

    name_to_gender = json.load(open('data/gender_predictions_10000.json', 'r'))

    unique_names = filtered_df['strip_display_name'].unique()

    chunk_size = 500
    for i in range(len(name_to_gender), len(unique_names), chunk_size):
        chunk = unique_names[i:i + chunk_size]
        print(
            f"Processing chunk {i // chunk_size + 1} of {len(unique_names) // chunk_size + 1}")
        predicted_genders = get_gender_predictions(chunk)
        name_to_gender.update(dict(zip(chunk, predicted_genders)))
        if (i // chunk_size) % 10 == 0:
            print(f"Saving progress after processing {i} names.")
            with open(f'data/gender_predictions_{i}.json', 'w') as f:
                json.dump(name_to_gender, f)
    filtered_df['predicted_gender'] = filtered_df['strip_display_name'].map(
        name_to_gender)
    gender_df = filtered_df[filtered_df['predicted_gender']
                            != 'Unknown'][['id', 'predicted_gender']]

In [ ]:
cited_by_count_df = df[['id', 'cited_by_count']]
works_api_url_df = df[['id', 'works_api_url']]
display_name_df = df[['id', 'display_name']]

In [ ]:
# from re import X

# dfs = [counts_by_year_df, x_concepts_df, gender_df,
#        cited_by_count_df, works_api_url_df, display_name_df]
# for _ in dfs:
#     if not os.path.exists(f"data/{_.columns[1]}.jsonl"):
#         _.to_json(f"data/{_.columns[1]}.jsonl", orient='records', lines=True)

In [ ]:
# import json


# def convert_json_to_jsonl(input_file, output_file):
#     # Open the JSON file and load the data
#     with open(input_file, 'r') as f:
#         data = json.load(f)

#     # Ensure the data is a list of dictionaries
#     if isinstance(data, dict):
#         data = [data]

#     # Open the output file in write mode
#     with open(output_file, 'w') as f_out:
#         for entry in data:
#             # Convert each dictionary to a JSON string and write it to the file
#             json_line = json.dumps(entry)
#             f_out.write(json_line + '\n')


# # Usage
# input_json_file = 'data.json'
# output_jsonl_file = 'data.jsonl'
# convert_json_to_jsonl(input_json_file, output_jsonl_file)

In [ ]:
publications_df = pd.read_json('data/author_publications.json')

In [ ]:
import requests

result = requests.get(df[:1]['works_api_url'].iloc[0]).json()

In [ ]:
import os
import logging
from sys import stdout

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s', force=True, stream=stdout)

chunk_size = 1000
publications = []

for start in range(0, len(df), chunk_size):
    end = start + chunk_size
    chunk = df.iloc[start:end]
    logging.info(f"Processing chunk from {start} to {end}")
    for index, row in chunk.iterrows():
        response = requests.get(row['works_api_url'], params={
                                'api_key': os.getenv('OPENALEX_API_KEY')}).json()
        result = response['results']
        if len(result) == 0:
            logging.info(f"No results found for row {index}")
            break
        for i in result:
            publications.append(i)

    # Save progress periodically in JSON Lines format
    with open(f'data/publications_chunk_{start//chunk_size}.jsonl', 'w') as f:
        for publication in publications:
            f.write(json.dumps(publication) + '\n')
    logging.info(f"Saved publications chunk {start//chunk_size}")

In [ ]:
import glob
import os

# Save all publications to a single JSON Lines file
with open('data/publications_list.jsonl', 'w') as outfile:
    for publication in publications:
        outfile.write(json.dumps(publication) + '\n')

# Delete the chunk files
for chunk_file in glob.glob('data/publications_chunk_*.jsonl'):
    os.remove(chunk_file)